# Security

We live in a connected world. This chapter gives the an insight to the security aspects of the _DashT_ overlay plug-in for the popular [OpenCPN](https://opencpn.org) chart plotter. It presents the software security policy and explain its implementation.

>**TIP**: Even if you are not that much of interested in this type of talk, please quickly take a look at the [data flow diagram](#Data-flow) and you can spot out is your boat's infrastructure concerned with any potential security issues. If it is, you have certainly already given a thought to the security aspects and can adapt _DashT_, accordingly. See also: [Privacy](./privacy.ipynb#Privacy).

## Data flow

The below diagram depicts the data flow in and out from the overlay functions. 

<img src="img/s_805_DashT_Security_Dataflow_Diagram.png"
alt="DashT Security - Dataflow diagram " width="600">[(zoom)](img/805_DashT_Security_Dataflow_Diagram.png)

_DashT_ C++ (plug-in) part **receives data** from following sources:

1. From OpenCPN
  - NMEA-0183 messages ;
  - GPS fix and magnetic deviation ;
  - Routing and waypoint change information.
2. From Signal K server node
  - Signal K delta channel by subscription - nothing if no instruments subscribing.

_DashT_ C++ (plug-in) part **sends data** to following destinations:

1. To OpenCPN
  - Route and waypoint creation and modification requests via the ABI ;
  - Storage of data from historical instruments into the files under the responsibility of OpenCPN ;
  - Storage of all data from all sources into a InfluxDB line data file under the responsibility of OpenCPN.
2. To InfluxDB v2 Time Series database
  - All received data or a selection of that data.

_DashT_ TypeScript/JavaScript (web based instruments) part **receives data** from following sources:

1. From InfluxDB v2 Time Series database
  - Read back any selected data from the data written into it by the C++ (plug-in) part.

## Security Policy

The security has an uttermost priority. The user's computer or infrastructure security shall not been compromised or the system integrity threatened - not directly or indirectly - because he or she is using _DashT_ plug-in.

The security implementation of _DashT_ shall meet the [SMART criteria](https://en.wikipedia.org/wiki/SMART_criteria), taking into account the surrounding FOSS and commercial ecosystem and putting those in balance with the very limited development and maintenance resources.

The achievability shall be described by an implementation plan.

The implementation plan shall clearly define the interfaces, to demonstrate how those "doors" are kept closed and how the own code base is kept safe.

The implementation plan shall demonstrate constant measurability by describing automated inspection against the currently applicable advisory.

The security threat shall be communicated to the end user via [GitHub project security page](https://github.com/canne/dashboard_tactics_pi/security) by publishing a security advisory.

It shall be under each end user's own responsibility to follow the security advisory.

The security fixes, if applicable are implemented only in the latest development version.

The security fixes or patches are no applied to earlier versions of this software.

User shall be therefore obliged to **upgrade to the latest published version or to an intermediate development version** to get the applied security fix if he or she deem necessary to do so.

There is **no guarantee in time** when the security fix will be applied; or that it will be applied at all.

## Security Implementation

Please see the [dedicated document in the software package's documentation](https://github.com/canne/dashboard_tactics_pi/blob/213c034292ef8a313e59350e6875ae30624ac83b/docs/security/security.ipynb) (_the link is pointing to dedicated document in_ [source code repository](https://github.com/canne/dashboard_tactics_pi), see `docs/security`).